# Python建模库简介

在本书中，我专注于为使用Python进行数据分析提供编程基础。由于数据分析师和科学家经常报告花费大量时间进行数据整理和准备，因此本书的结构反映了掌握这些技术的重要性。

您用于开发模型的库取决于应用程序。许多统计问题可以通过更简单的技术来解决，例如普通的最小二乘回归，而其他问题则可能需要更高级的机器学习方法。幸运的是，Python已成为实现分析方法的首选语言之一，因此在完成本书后，您可以探索许多工具。

在本章中，我将回顾熊猫的一些功能，这些功能在您与熊猫进行数据争吵以及模型拟合和评分之间来回切换时可能会有所帮助。然后，我将简要介绍两个流行的建模工具包statsmodels和scikit-learn。由于每个项目的规模都足以保证自己拥有一本专门的书，因此，我将不做任何详尽的工作，而是将您带到两个项目的在线文档以及其他一些基于Python的有关数据科学，统计和机器学习的书。

## 13.1 pandas与标准代码之间的接口

模型开发的常见工作流程是使用熊猫进行数据加载和清理，然后再切换到建模库以构建模型本身。模型开发过程的重要部分称为机器学习中的特征工程。这可以描述从原始数据集中提取信息的任何数据转换或分析，这些信息可能在建模环境中有用。我们在本书中探讨的数据聚合和GroupBy工具经常用于功能工程环境中。

尽管“好的”功能工程的详细信息不在本书的讨论范围之内，但我将展示一些方法，以使使用熊猫进行的数据处理和建模之间的切换尽可能轻松。

pandas与其他分析库之间的接触点通常是NumPy数组。要将DataFrame转换为NumPy数组，请使用.values属性：

In [1]:
import pandas as pd
import numpy as np

In [3]:
data = pd.DataFrame({'x0': [1, 2, 3, 4, 5],
                     'x1': [0.01, -0.01, 0.25, -4.1, 0.],
                     'y': [-1.5, 0., 3.6, 1.3, -2]})

In [4]:
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [5]:
data.columns

Index(['x0', 'x1', 'y'], dtype='object')

In [6]:
data.values

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

如您可能在前面的章节中所提到的，要转换回DataFrame，可以传递带有可选列名的二维ndarray：

In [7]:
df2 = pd.DataFrame(data.values, columns=['one', 'two', 'three'])

In [8]:
df2

,one,two,three
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,0.25,3.6
3,4.0,-4.10,1.3
4,5.0,0.00,-2.0


> .values属性适用于数据同质的情况，例如所有数字类型。如果您具有异构数据，则结果将是Python对象的ndarray：

In [10]:
df3 = data.copy()

In [11]:
df3['string'] = ['a', 'b', 'c', 'd', 'e']

In [12]:
df3

,x0,x1,y,string
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,c
3,4,-4.10,1.3,d
4,5,0.00,-2.0,e


In [13]:
df3.values

array([[1, 0.01, -1.5, 'a'],
       [2, -0.01, 0.0, 'b'],
       [3, 0.25, 3.6, 'c'],
       [4, -4.1, 1.3, 'd'],
       [5, 0.0, -2.0, 'e']], dtype=object)

对于某些模型，您可能只希望使用列的子集。我建议对值使用loc索引：

In [14]:
model_cols = ['x0', 'x1']

In [15]:
data.loc[:, model_cols].values

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

一些库具有对熊猫的本机支持，并且会自动为您完成一些工作：从DataFrame转换为NumPy，并将模型参数名称附加到输出表或Series的列。在其他情况下，您将必须手动执行此“元数据管理”。

在第12章中，我们研究了熊猫的分类类型和pandas.get_dummies函数。假设我们的示例数据集中有一个非数字列：

In [16]:
data['category'] = pd.Categorical(['a', 'b', 'a', 'a', 'b'], categories=['a', 'b'])

In [17]:
data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


如果我们想用虚拟变量替换“类别”列，我们创建虚拟变量，删除“类别”列，然后加入结果：

In [18]:
dummies = pd.get_dummies(data.category, prefix='category')

In [19]:
data_with_dummies = data.drop('category', axis=1).join(dummies)

In [20]:
data_with_dummies

,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,1,0
1,2,-0.01,0.0,0,1
2,3,0.25,3.6,1,0
3,4,-4.10,1.3,1,0
4,5,0.00,-2.0,0,1


使用虚拟变量拟合某些统计模型有一些细微差别。当您有不只简单的数字列时，使用Patsy（下一节的主题）可能会更简单且更不易出错。

## 13.2 使用Patsy创建模型描述

Patsy是一个Python库，用于描述统计模型（尤其是线性模型），该模型具有基于字符串的小型“公式语法”，其灵感来自于（但不完全是
与R和S统计编程语言使用的公式语法相同。

Patsy支持在statsmodels中指定线性模型，因此，我将重点介绍一些主要功能来帮助您启动和运行。Patsy的公式是一种特殊的字符串语法，如下所示：

```
    y ~ x0 + x1
```

